<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential()  # identity인 경우
        if stride != 1:  # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.tanh(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # (핵심) skip connection
        out = F.tanh(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes  # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.tanh(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset

data_transformations = {
    "basic": transforms.Compose([
        transforms.ToTensor()
    ]),
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["crop_flip"])
augmented_train_data = ConcatDataset((train_dataset, train_aug_dataset))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 75:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1328125
Current benign train loss: 2.2833914756774902

Current batch: 100
Current benign train accuracy: 0.2734375
Current benign train loss: 2.0246543884277344

Current batch: 200
Current benign train accuracy: 0.328125
Current benign train loss: 1.7533704042434692

Current batch: 300
Current benign train accuracy: 0.3828125
Current benign train loss: 1.7995858192443848

Total benign train accuarcy: 31.07
Total benign train loss: 731.9033223390579

[ Test epoch: 0 ]

Test accuarcy: 21.64
Test average loss: 0.02479565899372101
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.3515625
Current benign train loss: 1.8088129758834839

Current batch: 100
Current benign train accuracy: 0.3515625
Current benign train loss: 1.6913765668869019

Current batch: 200
Current benign train accuracy: 0.46875
Current benign train loss: 1.5125371217727661

Current batch: 300
Current benign train accur


Current batch: 200
Current benign train accuracy: 0.8203125
Current benign train loss: 0.5265666246414185

Current batch: 300
Current benign train accuracy: 0.859375
Current benign train loss: 0.4433579444885254

Total benign train accuarcy: 78.082
Total benign train loss: 244.3311653137207

[ Test epoch: 13 ]

Test accuarcy: 73.05
Test average loss: 0.00835770725607872
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.71875
Current benign train loss: 0.758908212184906

Current batch: 100
Current benign train accuracy: 0.8203125
Current benign train loss: 0.5754104256629944

Current batch: 200
Current benign train accuracy: 0.7421875
Current benign train loss: 0.7426134943962097

Current batch: 300
Current benign train accuracy: 0.8046875
Current benign train loss: 0.5491939187049866

Total benign train accuarcy: 79.086
Total benign train loss: 235.00966477394104

[ Test epoch: 14 ]

Test accuarcy: 71.61
Test average loss: 0.00875628331899643
Model S


Total benign train accuarcy: 84.248
Total benign train loss: 176.1209515631199

[ Test epoch: 26 ]

Test accuarcy: 67.24
Test average loss: 0.012259641683101653
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.84375
Current benign train loss: 0.44485071301460266

Current batch: 100
Current benign train accuracy: 0.8046875
Current benign train loss: 0.5739474296569824

Current batch: 200
Current benign train accuracy: 0.84375
Current benign train loss: 0.5021701455116272

Current batch: 300
Current benign train accuracy: 0.8203125
Current benign train loss: 0.47548019886016846

Total benign train accuarcy: 84.812
Total benign train loss: 170.19379799067974

[ Test epoch: 27 ]

Test accuarcy: 78.3
Test average loss: 0.006818611598014832
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.875
Current benign train loss: 0.3966352343559265

Current batch: 100
Current benign train accuracy: 0.8515625
Current benign train l


Current batch: 0
Current benign train accuracy: 0.84375
Current benign train loss: 0.39219605922698975

Current batch: 100
Current benign train accuracy: 0.8046875
Current benign train loss: 0.5804356336593628

Current batch: 200
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3132426142692566

Current batch: 300
Current benign train accuracy: 0.875
Current benign train loss: 0.32607629895210266

Total benign train accuarcy: 86.882
Total benign train loss: 147.84997214376926

[ Test epoch: 40 ]

Test accuarcy: 82.94
Test average loss: 0.00509338581264019
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.8984375
Current benign train loss: 0.32477399706840515

Current batch: 100
Current benign train accuracy: 0.9140625
Current benign train loss: 0.3115357458591461

Current batch: 200
Current benign train accuracy: 0.9140625
Current benign train loss: 0.313564270734787

Current batch: 300
Current benign train accuracy: 0.828125
Cur


Current batch: 200
Current benign train accuracy: 0.953125
Current benign train loss: 0.12748275697231293

Current batch: 300
Current benign train accuracy: 0.9453125
Current benign train loss: 0.1585913598537445

Total benign train accuarcy: 94.046
Total benign train loss: 67.23001762107015

[ Test epoch: 53 ]

Test accuarcy: 90.77
Test average loss: 0.0029544324517250063
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 0.9765625
Current benign train loss: 0.08053341507911682

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train loss: 0.10727500170469284

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.06763359904289246

Current batch: 300
Current benign train accuracy: 0.921875
Current benign train loss: 0.23518525063991547

Total benign train accuarcy: 94.31
Total benign train loss: 64.4663977175951

[ Test epoch: 54 ]

Test accuarcy: 90.71
Test average loss: 0.002923535093665123
Mo


Total benign train accuarcy: 95.584
Total benign train loss: 48.60422612167895

[ Test epoch: 66 ]

Test accuarcy: 90.07
Test average loss: 0.003249901293963194
Model Saved!

[ Train epoch: 67 ]

Current batch: 0
Current benign train accuracy: 0.96875
Current benign train loss: 0.09053216874599457

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train loss: 0.12197394669055939

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.06817831844091415

Current batch: 300
Current benign train accuracy: 0.90625
Current benign train loss: 0.20329587161540985

Total benign train accuarcy: 95.646
Total benign train loss: 46.76909635588527

[ Test epoch: 67 ]

Test accuarcy: 90.08
Test average loss: 0.003363241923600435
Model Saved!

[ Train epoch: 68 ]

Current batch: 0
Current benign train accuracy: 0.96875
Current benign train loss: 0.15931154787540436

Current batch: 100
Current benign train accuracy: 0.9609375
Current benign trai


Test accuarcy: 91.48
Test average loss: 0.0029338985353708266
Model Saved!

[ Train epoch: 80 ]

Current batch: 0
Current benign train accuracy: 0.9765625
Current benign train loss: 0.0805765688419342

Current batch: 100
Current benign train accuracy: 0.984375
Current benign train loss: 0.061191655695438385

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.0669979602098465

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.03501806780695915

Total benign train accuarcy: 97.998
Total benign train loss: 24.845517837442458

[ Test epoch: 80 ]

Test accuarcy: 91.54
Test average loss: 0.002969058449938893
Model Saved!

[ Train epoch: 81 ]

Current batch: 0
Current benign train accuracy: 0.9765625
Current benign train loss: 0.08101874589920044

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train loss: 0.07111487537622452

Current batch: 200
Current benign train accuracy: 1.0
Current ben


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.03280297666788101

Current batch: 200
Current benign train accuracy: 0.984375
Current benign train loss: 0.05157458782196045

Current batch: 300
Current benign train accuracy: 0.984375
Current benign train loss: 0.03895925357937813

Total benign train accuarcy: 98.356
Total benign train loss: 20.942505994811654

[ Test epoch: 93 ]

Test accuarcy: 91.57
Test average loss: 0.0030996224291622637
Model Saved!

[ Train epoch: 94 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.04495648294687271

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.03784150257706642

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.022302227094769478

Current batch: 300
Current benign train accuracy: 0.96875
Current benign train loss: 0.07201727479696274

Total benign train accuarcy: 98.272
Total benign train loss: 2